# Project: [Analyzing Motorcycle Part Sales](https://app.datacamp.com/learn/projects/1574)

You're working for a company that sells motorcycle parts, and they've asked with some help in analyzing their sales data!

They operate three warehouses in the area, selling both retail and wholesale. They offer a variety of parts and accept credit card, cash, and bank transfer as payment methods. However, each payment type incurs a different fee.

The board of directors want to gain a better understanding of wholesale revenue by product line, and how this varies month-to-month and across warehouses. You have been tasked with calculating net revenue for each product line, grouping results by month and warehouse. The results should be filtered so that only `"Wholesale"` orders are included.

They have provided you with access to their database, which contains the following table called `sales`:

| Column         | Data type | Description                                                                      |
|----------------|-----------|----------------------------------------------------------------------------------|
| `order_number` | `VARCHAR` | Unique order number.                                                             |
| `date`         | `DATE`    | Date of the order, from June to August 2021.                                     |
| `warehouse`    | `VARCHAR` | The warehouse that the order was made from&mdash; `North`, `Central`, or `West`. |
| `client_type`  | `VARCHAR` | Whether the order was `Retail` or `Wholesale`.                                   |
| `product_line` | `VARCHAR` | Type of product ordered.                                                         |
| `quantity`     | `INT`     | Number of products ordered.                                                      |
| `unit_price`   | `FLOAT`   | Price per product (dollars).                                                     |
| `total`        | `FLOAT`   | Total price of the order (dollars).                                              |
| `payment`      | `VARCHAR` | Payment method&mdash;`Credit card`, `Transfer`, or `Cash`.                       |
| `payment_fee`  | `FLOAT`   | Percentage of `total` charged as a result of the `payment` method.               |


Your query output should be presented in the following format:

| `product_line` | `month` | `warehouse` | `net_revenue` |
|----------------|---------|-------------|---------------|
| product_one    | ---     | ---         | ---           |
| product_one    | ---     | ---         | ---           |
| product_one    | ---     | ---         | ---           |
| product_one    | ---     | ---         | ---           |
| product_one    | ---     | ---         | ---           |
| product_one    | ---     | ---         | ---           |
| product_two    | ---     | ---         | ---           |
| ...            | ...     | ...         | ...           |

### Project Instructions
Create a query to return `product_line`, the `month` from date, displayed as 'June', 'July', and 'August', the `warehouse`, and `net_revenue`.

- `net_revenue` is calculated by getting the sum of total and multiplying by `1 - payment_fee`, rounding to two decimal places.
- You will need to filter `client_type` so that only `'Wholesale'` orders are returned.
- The results should first be sorted by `product_line` and month in ascending order, then by `net_revenue` in descending order.
- The final dataframe must be named `revenue_by_product_line`

### Setup our environment and load our data
- Will use a DuckDB in memory database
- Will create the `sales` table and use NUMERIC instead of FLOAT as DuckDB suggests this for monetary values.
- Import the `sales.csv` data into our `sales` table

In [1]:
%load_ext sql
# create an in memory duckdb connection
%sql duckdb:/// --alias sales_db
%config SqlMagic.autopandas = True
%config SqlMagic.displaycon = False

Connecting to 'sales_db'

Create our table

In [2]:
%sql DROP TABLE IF EXISTS sales;

,Success


In [3]:
%%sql
-- create sales table script for duckDB
CREATE TABLE sales (
    order_number VARCHAR, -- unique order number
    date DATE, -- date of order
    warehouse VARCHAR, -- wharehouse that the order was made from
    client_type VARCHAR, -- whether the order was Retail or Wholesale
    product_line VARCHAR, -- type of product ordered
    quantity INT, -- number of products ordered
    unit_price NUMERIC, -- price per product (dollars) must be numeric for monetary amounts in duckdb
    total NUMERIC, -- total price of the order (dollars)
    payment VARCHAR, -- payment method
    payment_fee NUMERIC -- percentage of total charged as a result of payment method.
);

,Success


Copy our data from the `sales.csv` file into the sales table.

In [4]:
%sql COPY sales FROM 'data/motorcycle_part_sales.csv' (AUTO_DETECT TRUE);

,Success


In [ ]:
# incase things go sideways with the import (ex, file is already open )
# %sql ROLLBACK

Lets check that we don't have any missing values

In [17]:
%sql SUMMARIZE sales;

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,order_number,VARCHAR,C1,W99,992,None,None,None,None,None,1000,0.0%
1,date,DATE,2021-06-01,2021-08-28,89,None,None,None,None,None,1000,0.0%
2,warehouse,VARCHAR,Central,West,3,None,None,None,None,None,1000,0.0%
3,client_type,VARCHAR,Retail,Wholesale,2,None,None,None,None,None,1000,0.0%
4,product_line,VARCHAR,Breaking system,Suspension & traction,6,None,None,None,None,None,1000,0.0%
5,quantity,INTEGER,1,40,18,9.395,9.659206762217861,4,7,10,1000,0.0%
6,unit_price,"DECIMAL(18,3)",10.030,66.620,884,30.32204,12.256488034397838,21,29,38,1000,0.0%
7,total,"DECIMAL(18,3)",10.350,2546.330,991,289.113,345.2275964463849,93,178,322,1000,0.0%
8,payment,VARCHAR,Cash,Transfer,3,None,None,None,None,None,1000,0.0%
9,payment_fee,"DECIMAL(18,3)",0.000,0.030,3,0.02202,0.011438988174242083,0,0,0,1000,0.0%


Inspect the first five rows of our data

In [18]:
%sql SELECT * FROM sales LIMIT 5;

,order_number,date,warehouse,client_type,product_line,quantity,unit_price,total,payment,payment_fee
0,N1,2021-06-01,North,Retail,Breaking system,9,19.29,173.61,Cash,0.00
1,N2,2021-06-01,North,Retail,Suspension & traction,8,32.93,263.45,Credit card,0.03
2,N3,2021-06-01,North,Wholesale,Frame & body,16,37.84,605.44,Transfer,0.01
3,N4,2021-06-01,North,Wholesale,Suspension & traction,40,37.37,1494.80,Transfer,0.01
4,N5,2021-06-01,North,Retail,Frame & body,6,45.44,272.61,Credit card,0.03


## Possible Solutions

### DuckDB solution using monthname
- With DuckDB we can use the [monthname](https://duckdb.org/docs/sql/functions/timestamp) function to automatically convert our date timestamp to the name of the month.
- The solution required by the project was for the monthname to be ordered alphabetically...but IMO it should be in calendar order. 
- The following query used [ANY_VALUE](https://duckdb.org/docs/sql/aggregates#general-aggregate-functions) so that we can order by the date without including it in the group by.

In [14]:
%%sql
SELECT 
  product_line,
  MONTHNAME(date) AS month_name,
  warehouse,
  ROUND(SUM(total * (1 - payment_fee)), 2) AS net_revenue,
FROM sales
WHERE client_type = 'Wholesale'
GROUP BY product_line, month_name, warehouse
ORDER BY product_line, ANY_VALUE(date) ASC, net_revenue DESC;

,product_line,month_name,warehouse,net_revenue
0,Breaking system,June,Central,3648.14
1,Breaking system,June,West,1200.64
2,Breaking system,June,North,1472.93
3,Breaking system,July,West,3030.39
4,Breaking system,July,Central,3740.94
5,Breaking system,July,North,2568.55
6,Breaking system,August,Central,3009.10
7,Breaking system,August,North,1753.19
8,Breaking system,August,West,2475.71
9,Electrical system,June,North,2002.30


### DuckDB solution using monthname
- With DuckDB we can use the [monthname](https://duckdb.org/docs/sql/functions/timestamp) function to automatically convert our date timestamp to the name of the month.
- The solution required by the project was for the monthname to be ordered alphabetically...

In [9]:
%%sql
SELECT 
  product_line,
  monthname(date) AS month_name,
  warehouse,
  ROUND(SUM(total * (1 - payment_fee)), 2) AS net_revenue,
FROM sales
WHERE client_type = 'Wholesale'
GROUP BY product_line, month_name, warehouse
ORDER BY product_line, month_name ASC, net_revenue DESC;

,product_line,month_name,warehouse,net_revenue
0,Breaking system,August,Central,3009.10
1,Breaking system,August,West,2475.71
2,Breaking system,August,North,1753.19
3,Breaking system,July,Central,3740.94
4,Breaking system,July,West,3030.39
5,Breaking system,July,North,2568.55
6,Breaking system,June,Central,3648.14
7,Breaking system,June,North,1472.93
8,Breaking system,June,West,1200.64
9,Electrical system,August,North,4673.99


### DuckDB solution using CASE to specify Month names


In [16]:
%%sql 
SELECT 
    product_line,
    CASE 
        WHEN month(date) = 6 THEN 'June'
        WHEN month(date) = 7 THEN 'July'
        WHEN month(date) = 8 THEN 'August'
    END AS month_name,
    warehouse,
    ROUND(SUM(total * (1 - payment_fee))::numeric, 2) AS net_revenue
FROM sales
WHERE client_type = 'Wholesale'
GROUP BY product_line, warehouse, month_name
ORDER BY product_line, month_name, net_revenue DESC

,product_line,month_name,warehouse,net_revenue
0,Breaking system,August,Central,3009.10
1,Breaking system,August,West,2475.71
2,Breaking system,August,North,1753.19
3,Breaking system,July,Central,3740.94
4,Breaking system,July,West,3030.39
5,Breaking system,July,North,2568.55
6,Breaking system,June,Central,3648.14
7,Breaking system,June,North,1472.93
8,Breaking system,June,West,1200.64
9,Electrical system,August,North,4673.99


### DuckDB solution using Subquery

In [11]:
%%sql
SELECT
  product_line,
  month_name,
  warehouse,
  ROUND(SUM(revenue_per_order), 2) AS net_revenue 
FROM (
    SELECT 
		product_line, 
		monthname(date) AS month_name, -- display the date value as "Month"
		warehouse, 
		total * (1.0 - payment_fee) AS revenue_per_order -- perform the revenue calculation on each order
 	FROM sales
 	WHERE client_type = 'Wholesale'
) AS subquery
GROUP BY product_line, month_name, warehouse
ORDER BY product_line, month_name, net_revenue DESC;

,product_line,month_name,warehouse,net_revenue
0,Breaking system,August,Central,3009.10
1,Breaking system,August,West,2475.71
2,Breaking system,August,North,1753.19
3,Breaking system,July,Central,3740.94
4,Breaking system,July,West,3030.39
5,Breaking system,July,North,2568.55
6,Breaking system,June,Central,3648.14
7,Breaking system,June,North,1472.93
8,Breaking system,June,West,1200.64
9,Electrical system,August,North,4673.99


### DuckDB solution using a CTE

In [12]:
%%sql 
WITH revenues AS (
	SELECT 
		product_line, 
		monthname(date) AS month_name, -- display the date value as "Month"
		warehouse, 
		total * (1.0 - payment_fee) AS revenue_per_order -- perform the revenue calculation on each order
	FROM sales
	WHERE client_type = 'Wholesale' -- filter for only 'Wholesale' orders
	)
SELECT
  product_line,
  month_name,
  warehouse,
  ROUND(SUM(revenue_per_order), 2) AS net_revenue -- convert to numeric and sum all the totals per product_line, month, warehouse
FROM revenues
GROUP BY product_line, month_name, warehouse
ORDER BY product_line, month_name, net_revenue DESC;

,product_line,month_name,warehouse,net_revenue
0,Breaking system,August,Central,3009.10
1,Breaking system,August,West,2475.71
2,Breaking system,August,North,1753.19
3,Breaking system,July,Central,3740.94
4,Breaking system,July,West,3030.39
5,Breaking system,July,North,2568.55
6,Breaking system,June,Central,3648.14
7,Breaking system,June,North,1472.93
8,Breaking system,June,West,1200.64
9,Electrical system,August,North,4673.99


Close our duck db connection by alias

In [13]:
%sql --close sales_db

Confirm we have no active connections

In [14]:
%sql -l

current,url,alias


# PostgreSQL solutions
Note: These will not work with DuckDB as not all functions in PostgreSQL are the same.

## My PostgreSQL solution

In [ ]:
-- Postgres solution will not work in DuckDB
SELECT
	product_line,
	TO_CHAR(date, 'FMMonth') AS month,
	warehouse,
	ROUND(SUM(total * (1 - payment_fee))::NUMERIC, 2) AS net_revenue
FROM sales
WHERE client_type = 'Wholesale'
GROUP BY product_line, month, warehouse
ORDER BY product_line ASC, month, net_revenue DESC;

Note some differences between FMMonth and Month with [TO_CHAR](https://app.datacamp.com/workspace/external-link?url=https%3A%2F%2Fwww.postgresql.org%2Fdocs%2F15%2Ffunctions-formatting.html)

In [41]:
/*  PostgreSQL related
 -- Interesting observations which I did not know about with respect to the TO_CHAR
 -- TO_CHAR(date, 'Month') has trailing blanks as per the docs:
 -- Month - full capitalized month name (blank-padded to 9 chars) 
*/

SELECT 
 	LENGTH(RTRIM(TO_CHAR(date, 'Month'))) as with_trim,
	LENGTH(TO_CHAR(date, 'Month')) as no_trim,
	LENGTH(TO_CHAR(date, 'FMMonth'))
FROM sales
WHERE client_type = 'Wholesale'
LIMIT 5;

,with_trim,no_trim,length
0,4,9,4
1,4,9,4
2,4,9,4
3,4,9,4
4,4,9,4


## PostgreSQL solution as provided by Datacamp

In [44]:
-- PostgreSQL solution as provided by Datacamp
SELECT product_line,
    CASE WHEN EXTRACT('month' from date) = 6 THEN 'June'
        WHEN EXTRACT('month' from date) = 7 THEN 'July'
        WHEN EXTRACT('month' from date) = 8 THEN 'August'
    END as month,
    warehouse,
    ROUND(SUM(total * (1 - payment_fee))::numeric, 2) AS net_revenue
FROM sales
WHERE client_type = 'Wholesale'
GROUP BY product_line, warehouse, month
ORDER BY product_line, month, net_revenue DESC

,product_line,month,warehouse,net_revenue
0,Breaking system,August,Central,3009.10
1,Breaking system,August,West,2475.71
2,Breaking system,August,North,1753.19
3,Breaking system,July,Central,3740.94
4,Breaking system,July,West,3030.39
5,Breaking system,July,North,2568.55
6,Breaking system,June,Central,3648.14
7,Breaking system,June,North,1472.93
8,Breaking system,June,West,1200.64
9,Electrical system,August,North,4673.99


## PostgreSQL solutions using a CTE

In [48]:
-- PostgreSQL Solution using a CTE
WITH revenues AS (
	SELECT 
		product_line, 
		TO_CHAR(date, 'FMMonth') AS month, -- display the date value as "Month"
		warehouse, 
		total * (1.0 - payment_fee) AS revenue_per_order -- perform the revenue calculation on each order
	FROM sales
	WHERE client_type = 'Wholesale' -- filter for only 'Wholesale' orders
	)
SELECT
  product_line,
  month,
  warehouse,
  ROUND( SUM(revenue_per_order) :: NUMERIC, 2) AS net_revenue -- convert to numeric and sum all the totals per product_line, month, warehouse
FROM revenues
GROUP BY product_line, month, warehouse
ORDER BY product_line, month, net_revenue DESC;

,product_line,month,warehouse,net_revenue
0,Breaking system,August,Central,3009.10
1,Breaking system,August,West,2475.71
2,Breaking system,August,North,1753.19
3,Breaking system,July,Central,3740.94
4,Breaking system,July,West,3030.39
5,Breaking system,July,North,2568.55
6,Breaking system,June,Central,3648.14
7,Breaking system,June,North,1472.93
8,Breaking system,June,West,1200.64
9,Electrical system,August,North,4673.99


In [49]:
-- PostgreSQL solution using a subquery
SELECT
  product_line,
  month,
  warehouse,
  ROUND( SUM(revenue_per_order) :: NUMERIC, 2) AS net_revenue -- convert to numeric and sum all the totals per product_line, month, warehouse

FROM (
	SELECT 
		product_line, 
		TO_CHAR(date, 'FMMonth') AS month, -- display the date value as "Month"
		warehouse, 
		total * (1.0 - payment_fee) AS revenue_per_order -- perform the revenue calculation on each order
 	FROM sales
 	WHERE client_type = 'Wholesale'
) AS subquery
GROUP BY product_line, month, warehouse
ORDER BY product_line, month, net_revenue DESC;

,product_line,month,warehouse,net_revenue
0,Breaking system,August,Central,3009.10
1,Breaking system,August,West,2475.71
2,Breaking system,August,North,1753.19
3,Breaking system,July,Central,3740.94
4,Breaking system,July,West,3030.39
5,Breaking system,July,North,2568.55
6,Breaking system,June,Central,3648.14
7,Breaking system,June,North,1472.93
8,Breaking system,June,West,1200.64
9,Electrical system,August,North,4673.99
